In [109]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools


init_notebook_mode(connected=True)

import pandas as pd

# Modules for widgets
import ipywidgets as wg
from IPython.display import display

# Modules for delay
import time
from collections import Counter

# Buttton to hide code, not selfwritten, only used to make the reading-experience better.
# Source: https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}


def toggle_code(state):

    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))


def button_action(value):

    """
    Calls the toggle_code function and updates the button description.
    """

    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]


state = False
toggle_code(state)

button = wg.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Show code')

In [110]:
df = pd.read_csv('globalterrorismdb_0718dist.csv', encoding = "ISO-8859-1", low_memory=False)

#df.fillna(0, inplace=True)

In [111]:
def topfig():
    countries = df['country_txt'].unique()
    killed = []
    wounded = []
    victims= []

    for country in countries:
        cur_yr = df[df['country_txt'] == country]
        wnd = cur_yr['nwound'].sum()
        kll = cur_yr['nkill'].sum()
        vctm = wnd + kll
        victims.append(vctm)
        wounded.append(wnd)
        killed.append(kll)

    sort_cty = [victims for _,victims in sorted(zip(victims, countries), reverse = True)]
    sort_vctm = sorted(victims, reverse = True)
    sort_kill = [victims for _,victims in sorted(zip(victims, killed), reverse = True)]
    sort_wnd = [victims for _,victims in sorted(zip(victims, wounded), reverse = True)]

    trace1 = go.Bar(
        x = sort_cty[:20],
        y = sort_kill[:20],
        name="Doden"
    )

    trace2 = go.Bar(
         x = sort_cty[:20],
         y = sort_wnd[:20],
         name="Gewonden"
     )

    plotlayout = go.Layout(
        title = go.layout.Title(
            text ='Landen met het meeste slachtoffers 1970-2017',
        ),
        xaxis=go.layout.XAxis(
            type = 'category',
            title = 'land',
        ),
        yaxis = go.layout.YAxis(
            title = 'Slachtoffers'
        ),
        barmode='stack'
    )

    fig = go.Figure(data=[trace1, trace2], layout=plotlayout)
    iplot(fig)

In [112]:
western_countries = ['Greece', 'United States', 'Italy', 'East Germany (GDR)', 'West Germany (FRG)', 'Switzerland', 'Spain', 'Ireland',
                     'United Kingdom', 'Netherlands', 'Belgium', 'Canada', 'Australia', 'Sweden', 'Poland', 'Austria', 'Czechoslovakia',
                    'France', 'Portugal', 'Cyprus', 'New Zealand', 'Andorra', 'Malta', 'Japan' ,'Denmark', 'Norway', 'Vatican City', 'Luxembourg',
                     'Iceland', 'Finland', 'Hungary','Germany', 'Lithuania', 'Slovenia', 'Czech Republic', 'Latvia', 'Estonia', 'Slovak Republic',
                    'International']

df['western_country'] = ['Yes' if x in western_countries else 'No' for x in df['country_txt']]
westers_dataframe = df.loc[(df['western_country']) == 'Yes']
NON_westers_dataframe = df.loc[(df['western_country']) == 'No']

In [113]:
def targets():
    westers_dataframe = df.loc[(df['western_country']) == 'Yes']

    count = {}
    for i in df['targtype1_txt'].unique():
        count[str(i)] = 0
    for i in westers_dataframe["targtype1_txt"]:
        if i in count:
            count[i] += 1

    overig = 0
    pop_list = []
    for i, j in count.items():
        if j <= (len(westers_dataframe['targtype1_txt'])/100)*2:
            pop_list.append(i)
            overig += j

    for key in pop_list:
        count.pop(key)

    count['Overig (onder 2%)'] = overig

    trace_west = {
        "hole": 0.8,
        'text': [i for i, j in count.items()],
        "textposition":'outside', #yes
        "showlegend": False, 
        "type": "pie", 
        "values": [j for i, j in count.items()],
        #'marker': {'colors': ['rgb(215, 11, 11)', 'rgb(0, 48, 240)', 'rgb(0, 0, 0)']}
    }
    
    NON_westers_dataframe = df.loc[(df['western_country']) == 'No']

    count = {}
    for i in df['targtype1_txt'].unique():
        count[str(i)] = 0
    for i in NON_westers_dataframe["targtype1_txt"]:
        if i in count:
            count[i] += 1

    overig = 0
    pop_list = []
    for i, j in count.items():
        if j <= (len(NON_westers_dataframe['targtype1_txt'])/100)*2:
            pop_list.append(i)
            overig += j

    for key in pop_list:
        count.pop(key)

    count['Overig (onder 2%)'] = overig

    trace_rest = {
      "hole": 0.8,
      'text': [i for i, j in count.items()],
      "textposition":'outside', #yes
      "showlegend": False,
      "type": "pie",
      "values": [j for i, j in count.items()],
      #'marker': {'colors': ['rgb(215, 11, 11)', 'rgb(0, 48, 240)', 'rgb(0, 0, 0)']}
    }

    layout = dict(
        title = "Doelen van aanslagen westerse landen (1970 - 2017)"
    )

    
    fig = go.Figure(data = [trace_west] , layout = layout)
    iplot(fig)
    layout = dict(
        title = "Doelen van aanslagen niet-westerse landen (1970 - 2017)"
    )
    fig = go.Figure(data = [trace_rest] , layout = layout)
    iplot(fig)

In [114]:
year = wg.IntSlider(
    value=2007,
    min=1970,
    max=2017,
    step=1,
    description='Year: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [115]:
def donut_maker2(year):
    dt = []
    cyear = df.loc[df['iyear'] == year]
    countries = cyear['country_txt'].unique()
    
    for country in countries:
        cur_ct = cyear[cyear['country_txt'] == country]
        wnd = cur_ct['nwound'].sum()
        kll = cur_ct['nkill'].sum()
        att = 0
        for row in cur_ct['country_txt']:
                att += 1
                
        if kll+wnd >= 25.0 or att >= 10:

            if country in western_countries:
                trace0 = go.Scatter(
                    x = [att],
                    y = [kll+wnd],
                    mode = 'markers',
                    marker = dict(
                        size = 10,
                        color = 'orange'
                    ),
                    name = country,
                    text = country
                )
                dt.insert(0, trace0)
            else:
                trace0 = go.Scatter(
                    x = [att],
                    y = [kll+wnd],
                    mode = 'markers',
                    marker = dict(
                        size = 10,
                        color = 'darkblue'
                    ),
                    name = country,
                    text = country
                )
                dt.append(trace0)

    layout = go.Layout(
        autosize = True,
        title = go.layout.Title(
            text ="Slachtoffers tegenover aanslagen (" + str(year) + ")",
        ),
        xaxis=go.layout.XAxis(
            type = 'log',
            title = 'Aanslagen',
            autorange = True,
        ),
        yaxis = go.layout.YAxis(
            type = 'log',
            title = 'Slachtoffers',
            autorange = True,
        ),
    )
    if dt == []:
        print("No data available for this time period.")
    fig = go.Figure(data= dt, layout=layout)
    iplot(fig)

In [116]:
def avgatt():
    ## West
    killed_west = westers_dataframe['nkill'].sum()
    wounded_west = westers_dataframe['nwound'].sum()
    attacks_west = len(westers_dataframe.index)

    avg_casualties_west = (killed_west+wounded_west)/attacks_west

    ## Rest of the world
    killed_not_west = NON_westers_dataframe['nkill'].sum()
    wounded_not_west = NON_westers_dataframe['nwound'].sum()
    attacks_not_west = len(NON_westers_dataframe.index)

    avg_casualties_not_west= (killed_not_west+wounded_not_west)/attacks_not_west

    trace1 = go.Bar(
        x = [''],
        y = [avg_casualties_west],
        marker = dict(
            color = 'orange'
        ),
        name="West"
    )

    trace2 = go.Bar(
        x = [''],
        y = [avg_casualties_not_west],
        marker = dict(
            color = 'darkblue'
        ),
        name="Rest of the world"
    )

    plotlayout = go.Layout(
        title = go.layout.Title(
            text ='Gemiddelde hoeveelheid slachtoffers per aanslag per regio (1970-2017)',
        ),
        xaxis=go.layout.XAxis(
            type = 'category',
            title = 'land',
            tickangle = 50
        ),
        yaxis = go.layout.YAxis(
            title = 'Average casualties'
        )
    )

    fig = go.Figure(data=[trace1, trace2], layout=plotlayout)
    iplot(fig)

In [117]:
def casualtiespl():
    countries = df['country_txt'].unique()
    killed = []
    wounded = []
    attacks = []

    for country in countries:
        cur_country = df[df['country_txt'] == country]
        wounded.append(cur_country['nwound'].sum())
        killed.append(cur_country['nkill'].sum())
        attacks.append(len(cur_country.index))
    
     ## West
    killed_west = westers_dataframe['nkill'].sum()
    wounded_west = westers_dataframe['nwound'].sum()
    attacks_west = len(westers_dataframe.index)

    avg_casualties_west = (killed_west+wounded_west)/attacks_west

    ## Rest of the world
    killed_not_west = NON_westers_dataframe['nkill'].sum()
    wounded_not_west = NON_westers_dataframe['nwound'].sum()
    attacks_not_west = len(NON_westers_dataframe.index)

    avg_casualties_not_west= (killed_not_west+wounded_not_west)/attacks_not_west
    

    avg_casualties = []
    casualties = []

    for i in range(len(countries)):
        cas = killed[i]+wounded[i]
        casualties.append(cas)
        avg_casualties.append(cas/attacks[i])

    sorted_indexes = list(range(len(avg_casualties)))
    sorted_indexes.sort(key=avg_casualties.__getitem__, reverse = False)

    sort_avg_casualties = list(map(avg_casualties.__getitem__, sorted_indexes))
    sort_countries = list(map(countries.__getitem__, sorted_indexes))
    sort_casualties = list(map(casualties.__getitem__, sorted_indexes))

    colours = []
    # define bar colours based on region
    for cnt in sort_countries:
        if cnt in western_countries:
            colours.append('orange')
        else:
            colours.append('darkblue')

    trace1 = go.Bar(
        y = sort_countries,
        x = sort_avg_casualties,
        name="Westerse landen",
        marker = dict(color = colours),
        orientation = 'h',
        width = 0.5
    )
    
    trace2 = go.Bar(
        y = [None],
        x = [None],
        name="Niet westerse landen",
        marker = dict(color = 'darkblue'),
        showlegend = True
    )

    average_west_trace = go.Scatter(
        y = [sort_countries[0], sort_countries[-1]],
        x = [avg_casualties_west, avg_casualties_west],
        name = 'Average western world',
        mode = 'lines',
        fill = 'tozerox',
        fillcolor = 'rgba(255, 165, 0, 0.25)',
        line = dict(color = 'orange')
    )

    average_rest_trace = go.Scatter(
        y = [sort_countries[0], sort_countries[-1]],
        x = [avg_casualties_not_west, avg_casualties_not_west],
        name = 'Average rest of the world',
        mode = 'lines',
        fill = 'tonextx',
        fillcolor = 'rgba(31, 119, 180, 0.25)',
        marker = dict(color = 'darkblue')
    )

    plotlayout = go.Layout(
        margin=go.layout.Margin(
            l=200,
            r=0
        ),
        height = 1000,
        title = go.layout.Title(
            text ='De hoeveelheid slachtoffers per aanslag per land in het westen en de rest van de wereld',
        ),
        yaxis=go.layout.YAxis(
            type = 'category',
            title = 'land',
            range = [len(countries)-40,len(countries)],
            tickangle = 0,
            dtick = 1,
            tickfont=dict(
            family='Old Standard TT, serif',
            size=11,
            color='black')
        ),
        xaxis = go.layout.XAxis(
            title = 'Average casualties'
        )
    )

    fig = go.Figure(data=[trace1, trace2, average_west_trace, average_rest_trace], layout=plotlayout)
    iplot(fig)

In [118]:
def mappl(df):
    # omzetten van niet meer bestaande landen
    df.loc[df['country'] == 499, 'country'] = 75
    df.loc[df['country_txt'] == 'East Germany (GDR)', 'country_txt'] = 'Germany'
    df.loc[df['country'] == 263, 'country'] = 75
    df.loc[df['country_txt'] == 'West Germany (FRG)', 'country_txt'] = 'Germany'
    df.loc[df['country'] == 359, 'country'] = 167
    df.loc[df['country_txt'] == 'Soviet Union', 'country_txt'] = 'Russia'
    df.loc[df['country'] == 406, 'country'] = 228
    df.loc[df['country_txt'] == 'South Yemen', 'country_txt'] = 'Yemen'
    df.loc[df['country'] == 377, 'country'] = 228
    df.loc[df['country_txt'] == 'North Yemen', 'country_txt'] = 'Yemen'
    df.loc[df['country'] == 236, 'country'] = 54
    df.loc[df['country_txt'] == 'Czechoslovakia', 'country_txt'] = 'Czech Republic'
    df.loc[df['country'] == 175, 'country'] = 1001
    df.loc[df['country_txt'] == 'Serbia-Montenegro', 'country_txt'] = 'Serbia'
    df.loc[df['country'] == 604, 'country'] = 605
    df.loc[df['country_txt'] == 'Zaire', 'country_txt'] = "People's Republic of the Congo"

    df1 = df.groupby('country_txt')['nkill'].sum()
    df1 = df1.reset_index()
    df2 = df.groupby('country_txt')['nwound'].sum()
    df2 = df2.reset_index()
    df3 = df['country_txt'].value_counts()
    df3 = df3.to_frame().sort_index()
    df3 = df3.reset_index()

    scl = [
        [0.0, '#D4FE8A'],
        [0.005, '#B1EC32'],
        [0.015, '#BBD936'],
        [0.030, '#C5C63A'],
        [0.05, '#CEB33E'],
        [0.07, '#D8A042'],
        [0.12, '#E28D46'],
        [0.25, '#EB7A4A'],
        [0.5, '#F5674E'],
        [1.0, '#FF5452']
    ]
    for col in df1.columns:
        df1[col] = df1[col].astype(str)
    for col in df2.columns:
        df2[col] = df2[col].astype(str)
    for col in df3.columns:
        df3[col] = df3[col].astype(str)

    df1['text'] = df1['country_txt'] + "<br>" + "Aantal Aanslagen: " + df3['country_txt']+ "<br>" + "Aantal Doden: " + df1['nkill'] + "<br>" + "Aantal Gewonden: " + df2['nwound']

    data = [go.Choropleth(
        colorscale = scl,
        autocolorscale = False,
        locations = df1['country_txt'],
        z = df1['nkill'],
        locationmode = 'country names',
        text = df1['text'],
        colorbar = go.choropleth.ColorBar(
            title = "Slachtoffers")
    )]

    layout = go.Layout(
        title = go.layout.Title(
            text = 'Doden door aanslagen in de Wereld (1970-2017)'
        ),
        geo = go.layout.Geo(
            scope = 'world',
            projection = go.layout.geo.Projection(type = 'natural earth'),
            showcountries = True),
    )

    fig = go.Figure(data = data, layout = layout)
    iplot(fig)
    df = pd.read_csv('globalterrorismdb_0718dist.csv', encoding = "ISO-8859-1", low_memory=False)

## De overdrijving van terrorisme in het westen.
Terrorisme is met hoge regelmaat in het nieuws. Het is een van de hot topics van de 21ste eeuw, echter hoort men alleen maar over de aanslagen in het westen. Het idee ontstaat dat het westen het meest te maken heeft met deze aanslagen, dit is niet zo. Met de volgende visualisaties zullen wij betogen dat terrorisme in het westen een veel minder groot probleem is dan in de rest van de wereld.

De gekozen data gaat over terroristische aanslagen tussen 1970 en 2017 over de hele wereld.

In [119]:
mappl(df)

De aanslagen in westerse landen (denk 9/11, de recente aanslag op nieuw zeeland, etc.) komen altijd groot in het nieuws, toch blijkt dat in de periode 1970-2017 er veel meer slachtoffers zijn gevallen in Irak, Afghanistan en Pakistan dan in de rest van de westerse wereld. Er is geen westers land dat een dodental boven de 5000 bereikt, terwijl in de rest van de wereld veel landen getallen boven de 20.000 bereiken.

In [120]:
topfig()

Kijken naar de top 20 landen op basis van slachtoffer schetst een niet heel ander beeld. De Verenigde Staten is het westerse land waar het meeste slachtoffers zijn gevallen, maar ook zij staan slechts op de 9e plek van de wereld, bijna buiten de top 10. Hiernaast valt het in de Verenigde Staten ook mee hoeveel doden er zijn gevallen in vergelijking tot de rest, er zijn veel meer gewonden, wat mogelijk te wijten is aan de sterke defensie van het land.

In [121]:
wg.interact(donut_maker2, year = year);

interactive(children=(IntSlider(value=2007, continuous_update=False, description='Year: ', max=2017, min=1970)…

In [122]:
avgatt()
casualtiespl()

Een vergelijking van het aantal slachtoffers tegenover de hoeveelheid aanslagen levert een interessant resultaat op. De westerse landen zitten gemiddeld onder de algemene curve die de niet westerse landen hebben. De westerse landen hebben dus niet alleen minder aanslagen maar ook minder slachtoffers per aanslag (± 3 voor de westerse landen en ± 5.5 voor de niet westerse landen). De betekent dus dat westerse landen niet alleen minder uitgekozen worden maar dat de aanslagen zelf ook veel minder destructief zijn.

In [123]:
targets()

De doelen van de aanslagen zou een goede verklaring kunnen geven van waarom er, ondanks het feit dat er minder slachtoffers zijn er toch een grotere interesse is in nieuws in aanslagen in het westen. Naast het feit dat het voor de westerse mensen interessanter is wat er in hun eigen land gaande is vindt er ook een verschil plaats in de doelwitten rond de aanslagen. In het westen is een groter aandeel van de aanslagen gericht op bedrijven, terwijl in de rest van de wereld de aanslagen meer op burgers zijn gericht. Aanslagen op bedrijven richten mogelijk meer schade aan, aangezien er mensen zonder baan door komen te zitten of doordat het het algemene dagelijks leven beïnvloedt. Dit verschil in focus kan tot de overdrijving van het terrorisme in het westen geleid hebben.


Terrorisme in de westerse wereld is overdreven in vergelijking tot de rest van de wereld. De extreme gevallen komen nog steeds niet eens in de buurt bij wat er in de rest van de wereld afspeelt. De reden dat de focus zo erg ligt op de westerse wereld is vanwege het feit dat het om onze omgeving gaat en door de focus van terroristische aanslagen, die meer gericht is op bedrijven dan burgers.